# **Forward Pass for FNN**

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

from tensorflow.python.framework.convert_to_constants import (
    convert_variables_to_constants_v2_as_graph
)

# ==========================================================
# USER INPUTS (ONLY THESE)
# ==========================================================

MODEL_PATH = r"D:\college\sem-8\models\FNN\asthma_fnn_binary.h5"
DATASET_PATH = r"D:\college\sem-8\dataset\clean_asthma_dataset.csv"
DATASET_FRACTION = 0.02
BATCH_SIZE = 32

# ==========================================================
# 1️⃣ LOAD MODEL (ANY KERAS MODEL)
# ==========================================================

model = tf.keras.models.load_model(MODEL_PATH, compile=False)
print("✔ Model loaded")

# Infer model input shape (remove batch dim)
model_input_shape = model.input_shape
if isinstance(model_input_shape, list):
    model_input_shape = model_input_shape[0]

feature_shape = tuple(model_input_shape[1:])
print("✔ Model input shape:", feature_shape)

# ==========================================================
# 2️⃣ LOAD DATASET (MODEL-AWARE)
# ==========================================================

def load_dataset(path, feature_shape):
    ext = os.path.splitext(path)[1].lower()

    # ---------- CSV / TABULAR ----------
    if ext == ".csv":
        df = pd.read_csv(path)

        num_features = int(np.prod(feature_shape))
        X = df.iloc[:, :num_features].values.astype(np.float32)

        # Dummy labels (not used for FLOPs)
        y = np.zeros(len(X), dtype=np.int32)

        return tf.data.Dataset.from_tensor_slices((X, y))

    # ---------- NUMPY ----------
    if ext == ".npz":
        data = np.load(path)
        return tf.data.Dataset.from_tensor_slices(
            (data["x"], data["y"])
        )

    raise ValueError("Unsupported dataset format")

dataset = load_dataset(DATASET_PATH, feature_shape)
dataset = dataset.batch(BATCH_SIZE)
print("✔ Dataset loaded")

# ==========================================================
# 3️⃣ SAMPLE DATASET (1–2%)
# ==========================================================

def sample_dataset(ds, fraction):
    total_batches = tf.data.experimental.cardinality(ds).numpy()
    sample_batches = max(1, int(total_batches * fraction))
    return ds.take(sample_batches), total_batches

sampled_ds, total_batches = sample_dataset(dataset, DATASET_FRACTION)

# ==========================================================
# 4️⃣ COMPUTE FLOPs (ROBUST & SAFE)
# ==========================================================

def compute_flops_per_sample(model, feature_shape):
    """
    Computes FLOPs for ONE forward pass of ONE sample
    """
    input_spec = tf.TensorSpec(
        shape=(1,) + feature_shape,
        dtype=tf.float32
    )

    concrete_fn = tf.function(model).get_concrete_function(input_spec)

    frozen_func, _ = convert_variables_to_constants_v2_as_graph(
        concrete_fn
    )

    flops = tf.compat.v1.profiler.profile(
        graph=frozen_func.graph,
        options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
    )

    return flops.total_float_ops

flops_per_sample = compute_flops_per_sample(model, feature_shape)
print("✔ FLOPs per sample:", flops_per_sample)

# ==========================================================
# 5️⃣ EXTRAPOLATE TO FULL DATASET
# ==========================================================

total_samples = total_batches * BATCH_SIZE
total_flops = flops_per_sample * total_samples
total_gflops = total_flops / 1e9

# ==========================================================
# 6️⃣ FINAL REPORT
# ==========================================================

print("\n========== FLOPs ESTIMATION REPORT ==========")
print(f"Total samples     : {total_samples}")
print(f"FLOPs per sample  : {flops_per_sample:.3e}")
print(f"Total FLOPs       : {total_flops:.3e}")
print(f"Total GFLOPs      : {total_gflops:.3f}")
print("============================================")


# **Forward Pass for CNN**

In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.python.framework.convert_to_constants import (
    convert_variables_to_constants_v2_as_graph
)

# ==========================================================
# USER INPUTS
# ==========================================================

MODEL_PATH = r"D:\college\sem-8\models\CNN\pothole_cnn_model.h5"
IMAGE_DIR  = r"D:\college\sem-8\dataset\pothole"   # class-wise folders
DATASET_FRACTION = 0.02
BATCH_SIZE = 16

# ==========================================================
# 1️⃣ LOAD CNN MODEL
# ==========================================================

model = tf.keras.models.load_model(MODEL_PATH, compile=False)
print("✔ Model loaded")

# Infer input shape (H, W, C)
model_input_shape = model.input_shape
if isinstance(model_input_shape, list):
    model_input_shape = model_input_shape[0]

feature_shape = tuple(model_input_shape[1:])
print("✔ Model input shape:", feature_shape)

IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = feature_shape

# ==========================================================
# 2️⃣ LOAD IMAGE DATASET (MODEL-AWARE)
# ==========================================================

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_DIR,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=False
)

print("✔ Dataset loaded")

# ==========================================================
# 3️⃣ SAMPLE DATASET (1–2%)
# ==========================================================

def sample_dataset(ds, fraction):
    total_batches = tf.data.experimental.cardinality(ds).numpy()
    sample_batches = max(1, int(total_batches * fraction))
    return ds.take(sample_batches), total_batches

sampled_ds, total_batches = sample_dataset(dataset, DATASET_FRACTION)

# ==========================================================
# 4️⃣ COMPUTE FLOPs PER SAMPLE
# ==========================================================

def compute_flops_per_sample(model, feature_shape):
    input_spec = tf.TensorSpec(
        shape=(1,) + feature_shape,
        dtype=tf.float32
    )

    concrete_fn = tf.function(model).get_concrete_function(input_spec)

    frozen_func, _ = convert_variables_to_constants_v2_as_graph(
        concrete_fn
    )

    flops = tf.compat.v1.profiler.profile(
        graph=frozen_func.graph,
        options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
    )

    return flops.total_float_ops

flops_per_sample = compute_flops_per_sample(model, feature_shape)
print("✔ FLOPs per sample:", flops_per_sample)

# ==========================================================
# 5️⃣ EXTRAPOLATE TO FULL DATASET
# ==========================================================

total_samples = total_batches * BATCH_SIZE
total_flops = flops_per_sample * total_samples
total_gflops = total_flops / 1e9

# ==========================================================
# 6️⃣ FINAL REPORT
# ==========================================================

print("\n========== CNN FLOPs ESTIMATION REPORT ==========")
print(f"Total samples     : {total_samples}")
print(f"FLOPs per sample  : {flops_per_sample:.3e}")
print(f"Total FLOPs       : {total_flops:.3e}")
print(f"Total GFLOPs      : {total_gflops:.3f}")
print("===============================================")
